In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
churn_training_df = pd.read_csv('churn_training.csv')
print(churn_training_df.shape)
print(churn_training_df.head())

(4999, 21)
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0           1  Female            0.0      No         No     1.0          Yes   
1           2    Male            1.0     Yes         No    11.0          Yes   
2           3    Male            0.0      No         No     1.0          Yes   
3           4  Female            0.0      No        Yes    43.0          Yes   
4           5  Female            0.0      No         No    54.0          Yes   

  MultipleLines InternetService       OnlineSecurity  ...  \
0            No              No  No internet service  ...   
1            No     Fiber optic                   No  ...   
2            No              No  No internet service  ...   
3            No             DSL                  Yes  ...   
4           Yes     Fiber optic                   No  ...   

      DeviceProtection          TechSupport          StreamingTV  \
0  No internet service  No internet service  No internet service   
1    

In [4]:
churn_training_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4999 entries, 0 to 4998
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        4999 non-null   int64  
 1   gender            4999 non-null   object 
 2   SeniorCitizen     4997 non-null   float64
 3   Partner           4998 non-null   object 
 4   Dependents        4997 non-null   object 
 5   tenure            4998 non-null   float64
 6   PhoneService      4999 non-null   object 
 7   MultipleLines     4999 non-null   object 
 8   InternetService   4998 non-null   object 
 9   OnlineSecurity    4999 non-null   object 
 10  OnlineBackup      4999 non-null   object 
 11  DeviceProtection  4998 non-null   object 
 12  TechSupport       4999 non-null   object 
 13  StreamingTV       4999 non-null   object 
 14  StreamingMovies   4998 non-null   object 
 15  Contract          4998 non-null   object 
 16  PaperlessBilling  4999 non-null   object 


In [9]:
churn_training_df.describe()

,customerID,SeniorCitizen,tenure,MonthlyCharges
count,4999.000000,4997.000000,4998.000000,4993.000000
mean,2500.000000,0.159696,32.265306,64.781965
std,1443.231328,0.366360,24.637326,30.145188
min,1.000000,0.000000,0.000000,18.250000
25%,1250.500000,0.000000,8.000000,35.450000
50%,2500.000000,0.000000,29.000000,70.400000
75%,3749.500000,0.000000,55.750000,89.950000
max,4999.000000,1.000000,72.000000,118.750000
